In [1]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import openpolicedata as opd

In [2]:
# Query for the entire table of available data
datasets = opd.datasets_query()
datasets.head()

,ID,State,SourceName,Jurisdiction,TableType,Year,Description,DataType,URL,LUT
0,11007091826889850715,Arizona,Gilbert,Gilbert,STOPS,MULTI,Standardized stop data from the Stanford Open ...,CSV,https://stacks.stanford.edu/file/druid:yg821jf...,{'date_field': 'date'}
1,16664737837317589007,Arizona,Mesa,Mesa,STOPS,MULTI,Standardized stop data from the Stanford Open ...,CSV,https://stacks.stanford.edu/file/druid:yg821jf...,{'date_field': 'date'}
2,15199773303449299559,Arizona,State Patrol,State Patrol,STOPS,MULTI,Standardized stop data from the Stanford Open ...,CSV,https://stacks.stanford.edu/file/druid:yg821jf...,{'date_field': 'date'}
3,3364584804382489640,Arkansas,Little Rock,Little Rock,TRAFFIC STOPS,MULTI,Standardized stop data from the Stanford Open ...,CSV,https://stacks.stanford.edu/file/druid:yg821jf...,{'date_field': 'date'}
4,18124027295945532046,California,Anaheim,Anaheim,TRAFFIC STOPS,MULTI,Standardized stop data from the Stanford Open ...,CSV,https://stacks.stanford.edu/file/druid:yg821jf...,{'date_field': 'date'}


In [3]:
# Query for all available datasets from Maryland
df = opd.datasets_query(state="Maryland")
df.head()

,ID,State,SourceName,Jurisdiction,TableType,Year,Description,DataType,URL,LUT
37,17055238351209082622,Maryland,Baltimore,Baltimore,ARRESTS,MULTI,Arrest in the City of Baltimore,ArcGIS,https://egis.baltimorecity.gov/egis/rest/servi...,{'date_field': 'ArrestDateTime'}
38,17040813124079383419,Maryland,Baltimore,Baltimore,CALLS FOR SERVICE,2017,Police Emergency and Non-Emergency calls to 911,ArcGIS,https://opendata.baltimorecity.gov/egis/rest/s...,{'date_field': 'calldatetime'}
39,6359901092225061889,Maryland,Baltimore,Baltimore,CALLS FOR SERVICE,2018,Police Emergency and Non-Emergency calls to 911,ArcGIS,https://opendata.baltimorecity.gov/egis/rest/s...,{'date_field': 'calldatetime'}
40,9645888771276616413,Maryland,Baltimore,Baltimore,CALLS FOR SERVICE,2019,Police Emergency and Non-Emergency calls to 911,ArcGIS,https://opendata.baltimorecity.gov/egis/rest/s...,{'date_field': 'calldatetime'}
41,4008793074328934800,Maryland,Baltimore,Baltimore,CALLS FOR SERVICE,2020,Police Emergency and Non-Emergency calls to 911,ArcGIS,https://opendata.baltimorecity.gov/egis/rest/s...,{'date_field': 'calldatetime'}


In [4]:
# Query for all stops data containing both traffic and investigatory stops in Virginia
df = opd.datasets_query(table_type=opd.TableTypes.STOPS, state="Virginia")
df.head()

,ID,State,SourceName,Jurisdiction,TableType,Year,Description,DataType,URL,LUT
103,6503611626068757982,Virginia,Virginia Community Policing Act,MULTI,STOPS,MULTI,A data collection consisting of all traffic an...,Socrata,data.virginia.gov,"{'id': '2c96-texw', 'date_field': 'incident_da..."


In [5]:
# To access the data, create a source using a Source Name (usually a police department name). There is an optional state input to clarify ambiguities.
# Create data source for the Virginia Community Policing Act

src = opd.Source(source_name="Virginia Community Policing Act", state="Virginia")
src.datasets.head()

,ID,State,SourceName,Jurisdiction,TableType,Year,Description,DataType,URL,LUT
103,6503611626068757982,Virginia,Virginia Community Policing Act,MULTI,STOPS,MULTI,A data collection consisting of all traffic an...,Socrata,data.virginia.gov,"{'id': '2c96-texw', 'date_field': 'incident_da..."


In [6]:
# Find out what types of data are available from this source

types = src.get_tables_types()

print(types)

['STOPS']


In [7]:
# Find out what years are available from the stops table

years = src.get_years(table_type=types[0])
print(years)

[2020, 2021, 2022]


In [8]:
# Find out what jurisdictions are contained in the stops table
jurisdictions = src.get_jurisdictions()
print(jurisdictions)

['Longwood University Police Department', 'Richard Bland College Police Department', "Prince William County Sheriff's Office", 'Shenandoah Police Department', "Fairfax County Sheriff's Office", "Dickenson County Sheriff's Office", "York - Poquoson Sheriff's Office", "Montgomery County Sheriff's Office", 'Woodrow Wilson Rehabilitation Center P.D.', 'Wytheville Police Department', 'Culpeper Police Department', "Alexandria City Sheriff's Office", 'Berryville Police Department', 'Purcellville Police Department', 'Radford University Police Department', 'Boones Mill Police Department', 'La Crosse Police Department', "Mecklenburg Sheriff's Office", 'Windsor Police Department', 'Henrico Police Department', "Westmoreland County Sheriff's Office", 'Colonial Heights Police Dept.', 'James City County Police Dept', 'Prince William County Police Department', 'Norton Police Department', 'Wise Police Department', 'Chesapeake Bay Bridge-Tunnel Police Dept.', 'Waynesboro Police Department', 'Fairfax Cit

In [9]:
# Find out what jurisdictions are contained in the stops table that containt he word "Arlington"
jurisdictions = src.get_jurisdictions(partial_name="Arlington")
print(jurisdictions)

["Arlington County Sheriff's Office", 'Arlington County Police Department']


In [10]:
# Load data from the source for the Arlington County Police Department in 2021
jurisdiction = "Arlington County Police Department"
t = src.load_from_url(year=2021, table_type=opd.TableTypes.STOPS, jurisdiction_filter=jurisdiction)

In [11]:
# Show the first 5 rows of the table
t.table.head(n=5)

,incident_date,agency_name,jurisdiction,reason_for_stop,race,ethnicity,age,gender,action_taken,violation_type,specific_violation,person_searched,vehicle_searched,additional_arrest
0,2021-01-01,Arlington County Police Department,ARLINGTON CO,OTHER,WHITE,HISPANIC,44,FEMALE,WARNING,LOCAL,14.2-16,NO,NO,NO
1,2021-01-01,Arlington County Police Department,ARLINGTON CO,EQUIPMENT VIOLATION,WHITE,NON-HISPANIC,23,MALE,WARNING,COMMONWEALTH,46.2-1030,NO,NO,NO
2,2021-01-01,Arlington County Police Department,ARLINGTON CO,TRAFFIC VIOLATION,BLACK OR AFRICAN AMERICAN,NON-HISPANIC,27,MALE,WARNING,COMMONWEALTH,46.2-1030,NO,NO,NO
3,2021-01-01,Arlington County Police Department,ARLINGTON CO,EQUIPMENT VIOLATION,WHITE,NON-HISPANIC,53,MALE,CITATION/SUMMONS,COMMONWEALTH,46.2-301,NO,NO,NO
4,2021-01-01,Arlington County Police Department,ARLINGTON CO,EQUIPMENT VIOLATION,WHITE,NON-HISPANIC,53,MALE,WARNING,COMMONWEALTH,46.2-612,NO,NO,NO


In [12]:
# Save to CSV file with default filename. Need double forward slashes in Windows.
output_dir = "..\\Data"
t.to_csv(outputDir=output_dir)

# For custom filename
# t.to_csv(outputDir="..//Data", filename="custome_filename.csv") 

In [13]:
# Load the CSV file back in

# Do not really need to create a new source. Could use t instead
new_src = opd.Source(source_name="Virginia Community Policing Act")
new_t = new_src.load_from_csv(outputDir=output_dir, year=2021, jurisdiction_filter=jurisdiction)

In [14]:
new_t.table.head()

,incident_date,agency_name,jurisdiction,reason_for_stop,race,ethnicity,age,gender,action_taken,violation_type,specific_violation,person_searched,vehicle_searched,additional_arrest
0,2021-01-01,Arlington County Police Department,ARLINGTON CO,OTHER,WHITE,HISPANIC,44,FEMALE,WARNING,LOCAL,14.2-16,NO,NO,NO
1,2021-01-01,Arlington County Police Department,ARLINGTON CO,EQUIPMENT VIOLATION,WHITE,NON-HISPANIC,23,MALE,WARNING,COMMONWEALTH,46.2-1030,NO,NO,NO
2,2021-01-01,Arlington County Police Department,ARLINGTON CO,TRAFFIC VIOLATION,BLACK OR AFRICAN AMERICAN,NON-HISPANIC,27,MALE,WARNING,COMMONWEALTH,46.2-1030,NO,NO,NO
3,2021-01-01,Arlington County Police Department,ARLINGTON CO,EQUIPMENT VIOLATION,WHITE,NON-HISPANIC,53,MALE,CITATION/SUMMONS,COMMONWEALTH,46.2-301,NO,NO,NO
4,2021-01-01,Arlington County Police Department,ARLINGTON CO,EQUIPMENT VIOLATION,WHITE,NON-HISPANIC,53,MALE,WARNING,COMMONWEALTH,46.2-612,NO,NO,NO
